## Bokeh

Bokeh: https://docs.bokeh.org/en/latest/

"Bokeh is a Python library for creating interactive visualizations for modern web browsers. It helps you build beautiful graphics, ranging from simple plots to complex dashboards with streaming datasets. With Bokeh, you can create JavaScript-powered visualizations without writing any JavaScript yourself."

* Originally funded by DARPA
* Produces JSON files which work as input for Javascript, which in turn are used to present data to a web browser
* Aims to help anyone who would like to quickly and easily connect powerful PyData tools to interactive plots, dashboards, and data applications.
* High-performance interactivity over very large or streaming datasets

In [ ]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show, save
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.models.widgets import Slider
from bokeh.layouts import row
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

In [ ]:
mu = 0
sigma = 1
x = np.linspace(-10,10,200)
y = 1/sigma/np.sqrt(2*np.pi) * np.exp(-0.5*((x-mu)**2/sigma**2))

In [ ]:
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(x,y, size=15, line_color="navy", fill_color="orange", fill_alpha=0.5)

show(p) # show the results

In [ ]:
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a line renderer with x and y coordinates and linewidth
p.line(x,y, line_width=2)

show(p) # show the results

In [ ]:
# A bit of house-keeping
# In order to get a Bokeh server app running through the JupyterHub
# We'll need to configure things with the proxy again

import urllib
import os

def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
#     base_url = os.environ['EXTERNAL_URL']
    base_url = 'https://jupyter.idre.ucla.edu'
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url

In [ ]:
# from bokeh.io import show, output_notebook, push_notebook
# from bokeh.plotting import figure

# from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel, Column
# from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs

# from bokeh.layouts import column, row, WidgetBox
# from bokeh.palettes import Category20_16

# from bokeh.application.handlers import FunctionHandler
# from bokeh.application import Application

# output_notebook()

In [ ]:
mu = 0
sigma = 1

x = np.linspace(-10,10,200)
y = 1/sigma/np.sqrt(2*np.pi) * np.exp(-0.5*((x-mu)**2/sigma**2))

# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a line renderer with x and y coordinates
p.line(source={'x':x,'y':y})

show(p)

In [ ]:
mu = 0
sigma = 1

x = np.linspace(-10,10,200)
y = 1/sigma/np.sqrt(2*np.pi) * np.exp(-0.5*((x-mu)**2/sigma**2))

# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a line renderer with x and y coordinates
p.line(source={'x':x,'y':y})

# create selector element
mu_select = Slider(start = -5, end = 5, step = 1, value = 0, title = 'mean')

# Create a row layout
layout = row(mu_select, p)

show(layout)

In [ ]:
def revise_output(newout):
    mu = 0
    sigma = 1

    x = np.linspace(-10,10,200)
    y = 1/sigma/np.sqrt(2*np.pi) * np.exp(-0.5*((x-mu)**2/sigma**2))

    # create a new plot with default tools, using figure
    p = figure(plot_width=400, plot_height=400)

    # add a line renderer with x and y coordinates
    #p.line(source={'x':x,'y':y})
    # need an updateable data structure
    sourcedata = {'x': x, 'y': y}
    source = ColumnDataSource(data=sourcedata)
    p.line(source=source)

    # and function to handle the update when mu_select changes
    # the (attr, old, new) is a required function signature
    def update(attr, old, new):
        y = 1/sigma/np.sqrt(2*np.pi) * np.exp(-0.5*((x-mu_select.value)**2/sigma**2))
        newsourcedata = {'x': x, 'y': y}
        source.data.update(newsourcedata)

    # create selector element
    mu_select = Slider(start = -5, end = 5, step = 1, value = 0, title = 'mean')
    # and listener on the selection object
    mu_select.on_change('value', update)

    # Create a row layout
    layout = row(mu_select, p)

    #show(layout)
    newout.add_root(layout)

handler = FunctionHandler(revise_output)
app = Application(handler)
show(app, notebook_url=remote_jupyter_proxy_url)

Demo of scatter + histograms with selection:
https://demo.bokeh.org/selection_histogram